# 🚀 Google Colab Setup

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ogautier1980/sandbox-ml/blob/main/cours/02_metriques_evaluation/02_demo_metriques_classification.ipynb)

**Si vous exécutez ce notebook sur Google Colab**, exécutez la cellule suivante pour installer les dépendances.

In [ ]:
# Installation des dépendances (Google Colab uniquement)import sysIN_COLAB = 'google.colab' in sys.modulesif IN_COLAB:    print('📦 Installation des packages...')        # Packages ML de base    !pip install -q numpy pandas matplotlib seaborn scikit-learn        # Détection du chapitre et installation des dépendances spécifiques    notebook_name = '02_demo_metriques_classification.ipynb'  # Sera remplacé automatiquement        # Ch 06-08 : Deep Learning    if any(x in notebook_name for x in ['06_', '07_', '08_']):        !pip install -q torch torchvision torchaudio        # Ch 08 : NLP    if '08_' in notebook_name:        !pip install -q transformers datasets tokenizers        if 'rag' in notebook_name:            !pip install -q sentence-transformers faiss-cpu rank-bm25        # Ch 09 : Reinforcement Learning    if '09_' in notebook_name:        !pip install -q gymnasium[classic-control]        # Ch 04 : Boosting    if '04_' in notebook_name and 'boosting' in notebook_name:        !pip install -q xgboost lightgbm catboost        # Ch 05 : Clustering avancé    if '05_' in notebook_name:        !pip install -q umap-learn        # Ch 11 : Séries temporelles    if '11_' in notebook_name:        !pip install -q statsmodels prophet        # Ch 12 : Vision avancée    if '12_' in notebook_name:        !pip install -q ultralytics timm segmentation-models-pytorch        # Ch 13 : Recommandation    if '13_' in notebook_name:        !pip install -q scikit-surprise implicit        # Ch 14 : MLOps    if '14_' in notebook_name:        !pip install -q mlflow fastapi pydantic        print('✅ Installation terminée !')else:    print('ℹ️  Environnement local détecté, les packages sont déjà installés.')

# Chapitre 02 - Métriques de Classification

**Objectifs :**
- Calculer manuellement les métriques de classification (Precision, Recall, F1)
- Utiliser scikit-learn pour les métriques
- Visualiser la matrice de confusion
- Tracer et interpréter les courbes ROC et Precision-Recall
- Comprendre l'impact du seuil de classification

**Dataset :** Breast Cancer Wisconsin (classification binaire)

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.data  # type: ignoresets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    roc_curve,
    roc_auc_score,
    precision_recall_curve,
    average_precision_score,
    ConfusionMatrixDisplay
)

# Configuration des graphiques
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

## 1. Chargement et Exploration des Données

In [ ]:
# Chargement du dataset Breast Cancer
data = load_breast_cancer()
X = data.data  # type: ignore
y = data.target  # type: ignore  # 0 = malignant (cancer), 1 = benign (bénin)

print("Dataset Breast Cancer Wisconsin")
print(f"Nombre d'exemples : {X.shape[0]}")
print(f"Nombre de features : {X.shape[1]}")
print(f"\nClasses : {data.target  # type: ignore_names}")  # type: ignore
print(f"Distribution des classes :")
print(f"  - Malignant (0) : {(y == 0).sum()} ({(y == 0).sum() / len(y) * 100:.1f}%)")
print(f"  - Benign (1) : {(y == 1).sum()} ({(y == 1).sum() / len(y) * 100:.1f}%)")

In [ ]:
# Visualisation de la distribution des classes
plt.figure(figsize=(8, 5))
unique, counts = np.unique(y, return_counts=True)
plt.bar(['Malignant', 'Benign'], counts, color=['#FF6B6B', '#4ECDC4'])
plt.ylabel('Nombre d\'exemples')
plt.title('Distribution des Classes - Breast Cancer Dataset')
for i, (label, count) in enumerate(zip(['Malignant', 'Benign'], counts)):
    plt.text(i, count + 10, f'{count}\n({count/len(y)*100:.1f}%)', ha='center')
plt.tight_layout()
plt.show()

print("Constat : Les classes sont relativement équilibrées (63% / 37%)")
print("L'accuracy sera donc une métrique raisonnable (mais pas suffisante).")

## 2. Entraînement d'un Modèle Simple

In [ ]:
# Split train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # Préserve la distribution des classes
)

print(f"Train set : {X_train.shape[0]} exemples")
print(f"Test set : {X_test.shape[0]} exemples")
print(f"\nDistribution train : {np.bincount(y_train)}")
print(f"Distribution test : {np.bincount(y_test)}")

In [ ]:
# Entraînement d'un modèle Logistic Regression
model = LogisticRegression(max_iter=10000, random_state=42)
model.fit(X_train, y_train)

# Prédictions
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probabilités pour la classe 1 (benign)

print("Modèle entraîné avec succès !")
print(f"Nombre de prédictions : {len(y_pred)}")

## 3. Matrice de Confusion

La matrice de confusion est la base de toutes les métriques de classification.

In [ ]:
# Calcul de la matrice de confusion
cm = confusion_matrix(y_test, y_pred)

print("Matrice de Confusion :")
print(cm)
print("\nInterprétation :")
print(f"  TN (True Negative) = {cm[0, 0]} : Malignant correctement classifiés")
print(f"  FP (False Positive) = {cm[0, 1]} : Malignant classés comme Benign (dangereux !)")
print(f"  FN (False Negative) = {cm[1, 0]} : Benign classés comme Malignant")
print(f"  TP (True Positive) = {cm[1, 1]} : Benign correctement classifiés")

In [ ]:
# Visualisation de la matrice de confusion
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Matrice avec valeurs absolues
disp1 = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Malignant', 'Benign'])
disp1.plot(ax=axes[0], cmap='Blues', values_format='d')
axes[0].set_title('Matrice de Confusion (valeurs absolues)', fontsize=12)

# Matrice normalisée (pourcentages)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
disp2 = ConfusionMatrixDisplay(confusion_matrix=cm_normalized, display_labels=['Malignant', 'Benign'])
disp2.plot(ax=axes[1], cmap='Greens', values_format='.2%')
axes[1].set_title('Matrice de Confusion (normalisée)', fontsize=12)

plt.tight_layout()
plt.show()

print("\nAnalyse :")
print(f"  - Le modèle a correctement classifié {cm[0,0] + cm[1,1]} / {cm.sum()} exemples")
print(f"  - {cm[0, 1]} cas malignant ont été classés comme benign (DANGEREUX en médecine !)")
print(f"  - {cm[1, 0]} cas benign ont été classés comme malignant (faux positif, stress inutile)")

## 4. Calcul Manuel des Métriques

Avant d'utiliser scikit-learn, calculons les métriques à la main pour bien comprendre.

In [ ]:
# Extraction de TP, TN, FP, FN
TN = cm[0, 0]
FP = cm[0, 1]
FN = cm[1, 0]
TP = cm[1, 1]

# Calcul manuel des métriques
accuracy_manual = (TP + TN) / (TP + TN + FP + FN)
precision_manual = TP / (TP + FP) if (TP + FP) > 0 else 0
recall_manual = TP / (TP + FN) if (TP + FN) > 0 else 0
f1_manual = 2 * (precision_manual * recall_manual) / (precision_manual + recall_manual) if (precision_manual + recall_manual) > 0 else 0
specificity_manual = TN / (TN + FP) if (TN + FP) > 0 else 0

print("=" * 60)
print("MÉTRIQUES CALCULÉES MANUELLEMENT")
print("=" * 60)
print(f"Accuracy    = (TP + TN) / Total = ({TP} + {TN}) / {TP+TN+FP+FN} = {accuracy_manual:.4f}")
print(f"Precision   = TP / (TP + FP) = {TP} / ({TP} + {FP}) = {precision_manual:.4f}")
print(f"Recall      = TP / (TP + FN) = {TP} / ({TP} + {FN}) = {recall_manual:.4f}")
print(f"F1-Score    = 2 * (P * R) / (P + R) = {f1_manual:.4f}")
print(f"Specificity = TN / (TN + FP) = {TN} / ({TN} + {FP}) = {specificity_manual:.4f}")
print("=" * 60)

In [ ]:
# Vérification avec scikit-learn
print("=" * 60)
print("VÉRIFICATION AVEC SCIKIT-LEARN")
print("=" * 60)
print(f"Accuracy    (sklearn) = {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision   (sklearn) = {precision_score(y_test, y_pred):.4f}")
print(f"Recall      (sklearn) = {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score    (sklearn) = {f1_score(y_test, y_pred):.4f}")
print("=" * 60)
print("\n✅ Les calculs manuels correspondent parfaitement à scikit-learn !")

## 5. Rapport de Classification Complet

In [ ]:
# Rapport complet avec classification_report
print("\n" + "=" * 70)
print("RAPPORT DE CLASSIFICATION COMPLET")
print("=" * 70)
print(classification_report(y_test, y_pred, target_names=['Malignant', 'Benign']))

print("\nInterprétation :")
print("  - Support : nombre d'exemples réels pour chaque classe")
print("  - Macro avg : moyenne simple des métriques (classes équivalentes)")
print("  - Weighted avg : moyenne pondérée par le nombre d'exemples")

## 6. Courbe ROC et AUC

La courbe ROC montre le compromis entre TPR (Recall) et FPR pour différents seuils.

In [ ]:
# Calcul de la courbe ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = roc_auc_score(y_test, y_pred_proba)

# Visualisation
plt.figure(figsize=(10, 7))

# Courbe ROC
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.3f})')

# Ligne de référence (classifieur aléatoire)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Classifieur aléatoire (AUC = 0.5)')

# Point correspondant au seuil par défaut (0.5)
default_threshold_idx = np.argmin(np.abs(thresholds - 0.5))
plt.scatter(fpr[default_threshold_idx], tpr[default_threshold_idx], 
            color='red', s=100, zorder=5, label=f'Seuil 0.5 (FPR={fpr[default_threshold_idx]:.3f}, TPR={tpr[default_threshold_idx]:.3f})')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR) = 1 - Specificity', fontsize=12)
plt.ylabel('True Positive Rate (TPR) = Recall', fontsize=12)
plt.title('Courbe ROC - Breast Cancer Classification', fontsize=14, fontweight='bold')
plt.legend(loc="lower right", fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nAUC-ROC = {roc_auc:.4f}")
print("\nInterprétation de l'AUC :")
print("  - AUC = 0.50 : Classifieur aléatoire (inutile)")
print("  - AUC = 0.70 : Acceptable")
print("  - AUC = 0.80 : Bon")
print("  - AUC = 0.90 : Excellent")
print("  - AUC = 1.00 : Parfait (rare, souvent signe d'overfitting)")
print(f"\n➡️ Notre modèle : AUC = {roc_auc:.3f} (Excellent !)")

## 7. Courbe Precision-Recall

Plus informative que ROC pour les classes déséquilibrées.

In [ ]:
# Calcul de la courbe Precision-Recall
precision_curve, recall_curve, pr_thresholds = precision_recall_curve(y_test, y_pred_proba)
avg_precision = average_precision_score(y_test, y_pred_proba)

# Visualisation
plt.figure(figsize=(10, 7))

# Courbe PR
plt.plot(recall_curve, precision_curve, color='blue', lw=2, 
         label=f'PR curve (AP = {avg_precision:.3f})')

# Ligne de base (proportion de positifs)
baseline = (y_test == 1).sum() / len(y_test)
plt.plot([0, 1], [baseline, baseline], linestyle='--', color='red', lw=2,
         label=f'Baseline (proportion positifs = {baseline:.3f})')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall (TPR)', fontsize=12)
plt.ylabel('Precision', fontsize=12)
plt.title('Courbe Precision-Recall - Breast Cancer Classification', fontsize=14, fontweight='bold')
plt.legend(loc="lower left", fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nAverage Precision (AP) = {avg_precision:.4f}")
print("\nL'Average Precision est l'aire sous la courbe PR.")
print("Plus elle est proche de 1, meilleur est le modèle.")

## 8. Impact du Seuil de Classification

Le seuil par défaut est 0.5, mais on peut l'ajuster selon le contexte.

In [ ]:
# Tester différents seuils
thresholds_to_test = [0.3, 0.5, 0.7, 0.9]

results = []
for threshold in thresholds_to_test:
    y_pred_threshold = (y_pred_proba >= threshold).astype(int)
    
    acc = accuracy_score(y_test, y_pred_threshold)
    prec = precision_score(y_test, y_pred_threshold, zero_division=0)
    rec = recall_score(y_test, y_pred_threshold, zero_division=0)
    f1 = f1_score(y_test, y_pred_threshold, zero_division=0)
    
    results.append({
        'Threshold': threshold,
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1-Score': f1
    })

df_results = pd.DataFrame(results)
print("\n" + "=" * 80)
print("IMPACT DU SEUIL DE CLASSIFICATION")
print("=" * 80)
print(df_results.to_string(index=False))
print("\nObservations :")
print("  - Seuil bas (0.3) : Recall élevé (peu de FN) mais Precision faible (beaucoup de FP)")
print("  - Seuil haut (0.9) : Precision élevée (peu de FP) mais Recall faible (beaucoup de FN)")
print("  - Seuil médian (0.5) : Compromis équilibré")

In [ ]:
# Visualisation de l'impact du seuil
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
colors = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12']

for i, (metric, color) in enumerate(zip(metrics, colors)):
    ax = axes[i // 2, i % 2]
    ax.plot(df_results['Threshold'], df_results[metric], marker='o', 
            linewidth=2, markersize=8, color=color, label=metric)
    ax.set_xlabel('Seuil de classification', fontsize=11)
    ax.set_ylabel(metric, fontsize=11)
    ax.set_title(f'Impact du seuil sur {metric}', fontsize=12, fontweight='bold')
    ax.grid(alpha=0.3)
    ax.set_ylim([0, 1.05])
    
    # Marquer le seuil par défaut (0.5)
    default_value = df_results[df_results['Threshold'] == 0.5][metric].values[0]
    ax.axvline(x=0.5, color='red', linestyle='--', alpha=0.5, label='Seuil par défaut')
    ax.scatter([0.5], [default_value], color='red', s=100, zorder=5)
    ax.legend(fontsize=9)

plt.tight_layout()
plt.show()

print("\nConclusion :")
print("Le choix du seuil dépend du contexte métier :")
print("  - Détection de cancer : Privilégier Recall (ne pas manquer de malades) → seuil bas")
print("  - Filtrage spam : Privilégier Precision (ne pas perdre d'emails légitimes) → seuil haut")
print("  - Usage général : F1-Score maximal → seuil optimal autour de 0.5")

## 9. Comparaison ROC vs Precision-Recall

Affichons les deux courbes côte à côte pour mieux comprendre leurs différences.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Courbe ROC
axes[0].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC (AUC = {roc_auc:.3f})')
axes[0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Aléatoire')
axes[0].set_xlabel('False Positive Rate', fontsize=12)
axes[0].set_ylabel('True Positive Rate (Recall)', fontsize=12)
axes[0].set_title('Courbe ROC', fontsize=14, fontweight='bold')
axes[0].legend(loc="lower right")
axes[0].grid(alpha=0.3)

# Courbe PR
axes[1].plot(recall_curve, precision_curve, color='blue', lw=2, label=f'PR (AP = {avg_precision:.3f})')
axes[1].axhline(y=baseline, linestyle='--', color='red', lw=2, label=f'Baseline ({baseline:.3f})')
axes[1].set_xlabel('Recall', fontsize=12)
axes[1].set_ylabel('Precision', fontsize=12)
axes[1].set_title('Courbe Precision-Recall', fontsize=14, fontweight='bold')
axes[1].legend(loc="lower left")
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\nQuand utiliser ROC vs PR ?")
print("\n📊 Courbe ROC :")
print("  ✅ Classes relativement équilibrées")
print("  ✅ Les vrais négatifs (TN) sont importants")
print("  ✅ Vue d'ensemble de la performance")
print("\n📊 Courbe Precision-Recall :")
print("  ✅ Classes très déséquilibrées (ex: 1% de positifs)")
print("  ✅ On se concentre sur les positifs (ignore les TN)")
print("  ✅ Plus sensible aux changements pour la classe minoritaire")
print("  ✅ Exemples : détection de fraude, anomalies, maladies rares")

## 10. Résumé et Conclusions

### Métriques calculées pour notre modèle :

In [ ]:
# Résumé final
print("\n" + "=" * 80)
print("RÉSUMÉ DES MÉTRIQUES - BREAST CANCER CLASSIFICATION")
print("=" * 80)
print(f"\n🎯 Accuracy       : {accuracy_score(y_test, y_pred):.4f} ({accuracy_score(y_test, y_pred)*100:.2f}%)")
print(f"🎯 Precision      : {precision_score(y_test, y_pred):.4f} ({precision_score(y_test, y_pred)*100:.2f}%)")
print(f"🎯 Recall         : {recall_score(y_test, y_pred):.4f} ({recall_score(y_test, y_pred)*100:.2f}%)")
print(f"🎯 F1-Score       : {f1_score(y_test, y_pred):.4f}")
print(f"🎯 AUC-ROC        : {roc_auc:.4f}")
print(f"🎯 Avg Precision  : {avg_precision:.4f}")
print("\n" + "=" * 80)

print("\n✅ Points clés à retenir :")
print("\n1. La matrice de confusion est la base de toutes les métriques")
print("   TP, TN, FP, FN permettent de calculer toutes les autres métriques.")
print("\n2. L'accuracy peut être trompeuse avec des classes déséquilibrées")
print("   Toujours regarder Precision, Recall, F1 en complément.")
print("\n3. Le choix de la métrique dépend du contexte métier")
print("   - Médecine : privilégier Recall (ne pas manquer de malades)")
print("   - Spam : privilégier Precision (ne pas perdre d'emails importants)")
print("   - Général : F1-Score (compromis équilibré)")
print("\n4. AUC-ROC mesure la performance indépendamment du seuil")
print("   Utile pour comparer des modèles sans choisir de seuil.")
print("\n5. Courbe PR > ROC pour les classes déséquilibrées")
print("   Elle ignore les TN et se concentre sur les positifs.")
print("\n6. Le seuil de classification (par défaut 0.5) peut être ajusté")
print("   Selon que l'on privilégie Precision ou Recall.")

## Exercices

1. Modifiez le seuil de classification pour maximiser le F1-score. Quel seuil obtenez-vous ?

2. Créez un dataset déséquilibré (95% / 5%) et comparez les courbes ROC et PR.

3. Implémentez vous-même les fonctions `precision_score()`, `recall_score()` et `f1_score()` en NumPy pur.

4. Tracez la courbe du F1-score en fonction du seuil et trouvez le seuil optimal.

5. Calculez la matrice de confusion pour un problème multi-classes (ex: Iris dataset).